# Load everything

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import findspark  # Get rid of this in DataBricks
findspark.init()  # Get rid of this in DataBricks
from pyspark.sql import Row
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.types import FloatType
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType

from pyspark.sql.functions import udf
from pyspark.sql import functions as F 
from pyspark.sql.functions import explode, col, udf, mean as _mean, stddev as _stddev, log, log10
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.functions import lit

from pyspark.ml.feature import StringIndexer

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation


from pyspark.ml.tuning import CrossValidator, ParamGridBuilder



from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorSlicer


In [3]:
from tqdm import tqdm

In [4]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.functions import vector_to_array


In [5]:
config = SparkConf().setAll([('spark.executor.memory', '30g'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','8g')])
config.setAppName("proj")
config.set("spark.dynamicAllocation.minExecutors", "2");
config.set("spark.dynamicAllocation.maxExecutors", "2");
config.set("spark.dynamicAllocation.initialExecutors", "2"); # the number must be between the min and max
sc = SparkContext(conf=config)  # start a new sc with the current config
spark = SparkSession(sc)
sqlc=SQLContext(sc)
print(sc.getConf().getAll())  # print all the configuration

[('spark.dynamicAllocation.initialExecutors', '2'), ('spark.executor.memory', '30g'), ('spark.executor.id', 'driver'), ('spark.dynamicAllocation.minExecutors', '2'), ('spark.app.id', 'local-1607272611071'), ('spark.driver.host', '172.18.40.157'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.dynamicAllocation.maxExecutors', '2'), ('spark.rdd.compress', 'True'), ('spark.driver.memory', '8g'), ('spark.driver.port', '33887'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true'), ('spark.app.name', 'proj')]


In [1]:
!cat train_features_*.csv > train_feats.csv

In [6]:
df_train = spark.read.csv('train_feats.csv', header='true', inferSchema= 'true')   # path in HDFS file system

In [7]:
df_label = spark.read.csv('train_targets_scored.csv', header='true', inferSchema= 'true')

In [8]:
df = df_train.join(df_label, on=['sig_id'], how='left_outer')  # Jjoin them together

# Drop vechile

In [9]:
df = df.filter(df.cp_type == 'trt_cp')
df = df.drop('cp_type')

# OneHot

In [10]:
indexer = StringIndexer(inputCol="cp_dose", outputCol="cp_dose_cat")
df1 = indexer.fit(df).transform(df)
indexer = StringIndexer(inputCol="cp_time", outputCol="cp_time_cat")
df1 = indexer.fit(df1).transform(df1)
df1 = df1.drop('cp_dose')
df1 = df1.drop('cp_time')

encoder = OneHotEncoder(inputCols=["cp_time_cat", "cp_dose_cat"],
                        outputCols=["cp_time_onehot", "cp_dose_onehot"])

model = encoder.fit(df1)
df1 = model.transform(df1)
df1 = df1.withColumn("cp_time_cols", vector_to_array("cp_time_onehot")).select(df1.columns + [col("cp_time_cols")[i] for i in range(2)])
df1 = df1.withColumn("cp_dose_cols", vector_to_array("cp_dose_onehot")).select(df1.columns + [col("cp_dose_cols")[i] for i in range(1)])
df1 = df1.drop('cp_dose_cat',
 'cp_time_cat',
 'cp_time_onehot',
 'cp_dose_onehot',
)


# Feature Engineering  (credit to Jeff)

In [11]:
gene_feature_names = [name for name in df1.columns if 'g-' in name]
cell_feature_names =  [name for name in df1.columns if 'c-' in name]

df2 = df1.withColumn("gene_max", F.greatest(*gene_feature_names))
df2 = df2.withColumn("gene_min", F.least(*gene_feature_names))
df2 = df2.withColumn("cell_max", F.greatest(*cell_feature_names))
df2 = df2.withColumn("cell_min", F.least(*cell_feature_names))



# Drop high correlation features

In [15]:
feature_columns = gene_feature_names + cell_feature_names  # This came from the previous section
vectorAssembler = VectorAssembler(inputCols = feature_columns, outputCol = 'feats' )
feature_vector = vectorAssembler.transform(df2).select("feats")
# pyspark implementation of determining the correlations
corr_matrix = Correlation.corr(feature_vector, "feats").head()[0]

# Convert the correlation desne matrix and apply mask and to get the indicies where high correlations are observed
# In here, I convert the correlation matrix to numpy, and then use numpy's mask to obtain the lower traingle of the
# matrix. I used numpy becasue pyspark does not have mask.

corr_Array = corr_matrix.toArray()
masked_corr = np.ma.masked_where(np.triu(np.ones_like(corr_Array, dtype=bool)), corr_Array, copy=True)  
idx_high_corr_feats = set(np.argwhere((masked_corr > 0.90) | (masked_corr < -0.90) )[:,0])  # Set threshold to 90%

# Identify the column to drop and then drop it.
features_to_drop = np.array(feature_columns)[list(idx_high_corr_feats)].tolist()


# Finally

df3 = df2.drop(*features_to_drop)

In [17]:
print(features_to_drop)

['c-2', 'c-4', 'c-6', 'c-8', 'c-11', 'c-13', 'c-26', 'c-31', 'c-33', 'c-38', 'c-40', 'c-42', 'g-50', 'c-51', 'c-52', 'c-54', 'c-55', 'c-59', 'c-60', 'c-62', 'c-63', 'c-66', 'c-72', 'c-73', 'c-75', 'c-81', 'c-82', 'c-83', 'c-84', 'c-85', 'c-90', 'c-91', 'c-92', 'c-93', 'c-94', 'c-96']


# Feature trans

???

# Training

In [21]:
final_feature_names = list(set(df3.columns) - set(df_label.columns))

vectorAssembler = VectorAssembler(inputCols = final_feature_names, outputCol = 'feats' )
df4 = vectorAssembler.transform(df3).drop(*final_feature_names)

In [ ]:
df4

In [42]:
def train_individual_label(df, label_name):
    
    
    if df.filter(df[label_name] == 1).count() >= 2:
    
        temp_df = df.select('feats', label_name)

        # stratify split of the dataframe for train-test split
        seed = 42
        fractions = {1: 0.8, 0: 0.8}
        train_df = temp_df.stat.sampleBy(label_name, fractions, seed, )
        test_df =  temp_df.subtract(train_df)


        
        train_df = train_df.select(F.col(label_name).alias('label'), F.col('feats').alias('features'))  # Just renaming these columns
        test_df = test_df.select(F.col(label_name).alias('label'), F.col('feats').alias('features')) # Just renaming these columns
     


        # Feature selection using random forrest before using other models

        clf = RandomForestClassifier(numTrees=20, maxDepth=5,  seed=42)


        model = clf.fit(train_df)

        feature_importance = model.featureImportances.toArray()
        important_feature_idx = feature_importance.argsort()[-int(0.1 * len(feature_importance)):]  # Only get the top 10%, according to 
                                                                                                    # the feature importance from random forrest classifier



        # Now, after getting the index, filter the feature vector based on the above feature importance index

        slicer = VectorSlicer(inputCol="features", outputCol="sub_features", indices=important_feature_idx)
        final_train_df_sub_feats =  slicer.transform(train_df).drop('features')
        final_test_df_sub_feats = slicer.transform(test_df).drop('features')

        
        final_train_df_sub_feats = final_train_df_sub_feats\
                             .select(F.col('label'), F.col('sub_features').alias('features'))  # Just renaming these columns

        final_test_df_sub_feats = final_test_df_sub_feats\
                             .select(F.col('label'), F.col('sub_features').alias('features'))  # Just renaming these columns


        # Finally, use CV to train the model and get the best parameters

        lr = LogisticRegression(maxIter=10, )
        paramGrid = ParamGridBuilder() \
                        .addGrid(lr.regParam, [ 0.1, 0.01]) \
                        .addGrid(lr.elasticNetParam, [1,  0])\
                        .build()

        
        evaluator = MulticlassClassificationEvaluator( metricName='logLoss')
        crossval = StratifiedCrossValidator(estimator=lr,
                              estimatorParamMaps=paramGrid,
                              evaluator=evaluator,
                              numFolds=3)  

        cvModel = crossval.fit(final_train_df_sub_feats)
        prediction_df = cvModel.transform(final_test_df_sub_feats)

        cvModel.save(f"./logistics/{label_name}.model")
        prediction_df.write.save(f"./logistics/{label_name}_prediction_df.parquet", format="parquet")
        final_train_df_sub_feats.write.save(f"./logistics/{label_name}_train_df.parquet", format="parquet")

        log_loss = evaluator.evaluate(prediction_df)

        with open(f"./logistics/log.log", 'a') as f:
            f.write(str(log_loss) + '\n') 

        return (True, log_loss)
    else:
        return (False, None)

In [64]:
train_individual_label = None

Stratify 
ref: https://stackoverflow.com/questions/47637760/stratified-sampling-with-pyspark/47672336

In [44]:
with open(f"./logistics/log.log", 'w') as f:
    pass


temp_dict = {}
for name in tqdm(df_label.columns[1:]):
    temp_dict[name] = train_individual_label(df4,  name)

 26%|██▌       | 53/206 [10:17:06<29:41:28, 698.62s/it]


Py4JJavaError: An error occurred while calling o180392.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 14 in stage 27001.0 failed 1 times, most recent failure: Lost task 14.0 in stage 27001.0 (TID 418960, 172.18.40.157, executor driver): org.apache.hadoop.fs.FSError: java.io.IOException: Cannot allocate memory
	at org.apache.hadoop.fs.RawLocalFileSystem$LocalFSFileInputStream.read(RawLocalFileSystem.java:163)
	at java.base/java.io.BufferedInputStream.read1(BufferedInputStream.java:290)
	at java.base/java.io.BufferedInputStream.read(BufferedInputStream.java:351)
	at java.base/java.io.DataInputStream.read(DataInputStream.java:149)
	at org.apache.hadoop.fs.FSInputChecker.readFully(FSInputChecker.java:436)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.readChunk(ChecksumFileSystem.java:257)
	at org.apache.hadoop.fs.FSInputChecker.readChecksumChunk(FSInputChecker.java:276)
	at org.apache.hadoop.fs.FSInputChecker.read1(FSInputChecker.ja

In [43]:
with open(f"./logistics/log.log", 'w') as f:
    pass

train_individual_label(df4,  '5-alpha_reductase_inhibitor')

(True, 0.0025348745154193234)

In [40]:
# Reference: https://github.com/interviewstreet/spark-stratifier

import itertools
import numpy as np

from pyspark import since, keyword_only
from pyspark.ml import Estimator, Model
from pyspark.ml.common import _py2java
from pyspark.ml.param import Params, Param, TypeConverters
from pyspark.ml.param.shared import HasSeed
from pyspark.ml.tuning import CrossValidator, CrossValidatorModel
from pyspark.ml.util import *
from pyspark.ml.wrapper import JavaParams
from pyspark.sql.functions import rand
from functools import reduce
from multiprocessing.pool import ThreadPool

class StratifiedCrossValidator(CrossValidator):
  def stratify_data(self, dataset):
    """
    Returns an array of dataframes with the same ratio of passes and failures.

    Currently only supports binary classification problems.
    """

    epm = self.getOrDefault(self.estimatorParamMaps)
    numModels = len(epm)
    nFolds = self.getOrDefault(self.numFolds)
    split_ratio = 1.0 / nFolds

    passes = dataset[dataset['label'] == 1]
    fails = dataset[dataset['label'] == 0]

    pass_splits = passes.randomSplit([split_ratio for i in range(nFolds)])
    fail_splits = fails.randomSplit([split_ratio for i in range(nFolds)])
    for i in range(len(pass_splits)):
      one_count_train = pass_splits[i].count()
      zero_count_train = fail_splits[i].count()
      factor = zero_count_train/one_count_train
      if factor > 2:
        fail_splits[i] = fail_splits[i].sample(fraction = 0.5, withReplacement = False,  seed = 42) 
        
        pass_splits[i]  = pass_splits[i].sample(fraction = factor, withReplacement = True,  seed = 42)
        
        
#         pass_splits[i] = spark.createDataFrame(pass_splits[i].toPandas().sample(n = fail_splits[i].count(), replace = True,  random_state = 0))
    
    
    stratified_data = [pass_splits[i].unionAll(fail_splits[i]) for i in range(nFolds)]

    return stratified_data

  def _fit(self, dataset):
    est = self.getOrDefault(self.estimator)
    epm = self.getOrDefault(self.estimatorParamMaps)
    numModels = len(epm)
    eva = self.getOrDefault(self.evaluator)
    nFolds = self.getOrDefault(self.numFolds)
    seed = self.getOrDefault(self.seed)
    metrics = [0.0] * numModels
    
    stratified_data = self.stratify_data(dataset)
    
    for i in range(nFolds):
      train_arr = [x for j,x in enumerate(stratified_data) if j != i]
      train = reduce((lambda x, y: x.unionAll(y)), train_arr)
      validation = stratified_data[i]

      models = est.fit(train, epm)

      for j in range(numModels):
        model = models[j]
        metric = eva.evaluate(model.transform(validation, epm[j]))
        metrics[j] += metric/nFolds

    if eva.isLargerBetter():
      bestIndex = np.argmax(metrics)
    else:
      bestIndex = np.argmin(metrics)

    bestModel = est.fit(dataset, epm[bestIndex])
    return self._copyValues(CrossValidatorModel(bestModel, metrics))


# SCRAP

In [60]:
my_set = {'Geeks', 'for', 'geeks'} 
  
s = list(my_set)

TypeError: 'list' object is not callable

In [62]:
type(list)

list

In [ ]:
cell_feature_names

In [32]:
df.select("cp_type").distinct().show()

+-------+
|cp_type|
+-------+
| trt_cp|
+-------+



In [ ]:
fractions = df.select("x1").distinct().withColumn("fraction", lit(0.8)).rdd.collectAsMap()


In [6]:
from pyspark.sql.functions import lit


In [7]:
lit(0.8)

Column<b'0.8'>

In [8]:
from pyspark.sql.functions import lit
list = [(2147481832,23355149,1),(2147481832,973010692,1),(2147481832,2134870842,1),(2147481832,541023347,1),(2147481832,1682206630,1),(2147481832,1138211459,1),(2147481832,852202566,1),(2147481832,201375938,1),(2147481832,486538879,1),(2147481832,919187908,1),(214748183,919187908,1),(214748183,91187908,1)]
df = spark.createDataFrame(list, ["x1","x2","x3"])

In [9]:
df

DataFrame[x1: bigint, x2: bigint, x3: bigint]

In [10]:
fractions = df.select("x1").distinct().withColumn("fraction", lit(0.8)).rdd.collectAsMap()


In [12]:
fractions

{2147481832: 0.8, 214748183: 0.8}

In [15]:
df.select("x1").distinct().withColumn("fraction", lit(0.8)).show()

+----------+--------+
|        x1|fraction|
+----------+--------+
|2147481832|     0.8|
| 214748183|     0.8|
+----------+--------+



In [ ]:
temp_df = features_and_targets.withColumn('target_vector', (vector_to_string(array([features_and_targets[col] for col in target_names])))).select(['sig_id', 'target_vector'])
string_indexer = StringIndexer(inputCol = 'target_vector', outputCol = 'target')
string_indexer_model = string_indexer.fit(temp_df)
temp_df = string_indexer_model.transform(temp_df).drop('target_vector')

data = features_and_targets.join(temp_df, features_and_targets.sig_id == temp_df.sig_id, how = 'inner').drop(temp_df.sig_id)